# 7章　ニューラルネットワーク・ディープラーニング

1. 機械学習の種類３つ
2. 教師あり学習の基本
3. 単層パーセプトロン、ニューラルネットワーク、ディープラーニングの違い

# 7.1 ニューロンモデル

## 7.1.1 神経細胞
### 神経細胞が電気信号を送る仕組み
脳の中などの神経細胞は、軸索というケーブル（木の枝）のようなものでつながっていて、軸索からシナプスという継ぎ目を伝って電気信号を別の神経細胞に送る。\
ある神経細胞に送られてきた電気が細胞内に膜電位（細胞内の電位）として溜まっていき、一定値を超えるとまた次の神経細胞に電気が送られる。

### シナプスの仕組み
どれだけの電気信号を送るかは、シナプス伝達強度というもので決まる。\

- シナプス伝達強度が機械学習でいう重み\
- 神経細胞一つ一つが機械学習でいうノード

## 7.2.2 ニューロンモデル

説明変数２つあるとして、それらに重みをかけて総和を取り、0〜1の間の値で出力するニューロンモデルを考える。\

説明変数が2つあるから、入力総和を図示すると平面になる。\
これをシグモイド関数に通すから出力を0〜1に押しつぶした形になる。

範囲　6.3.3 - 7.1.2 OpenCV C-1 - C-2 \
担当　7.0 - 7.1.1とOpenCV C-1

8/8 c-3 - c-5